<a href="https://colab.research.google.com/github/mynhungg/Datamining/blob/main/PhoBERT_Emotion_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Prepare

## Connect to Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import os
path = "/content/drive/MyDrive/Colab Notebooks/PhoBERT_Emotion"
os.chdir(path)

In [ ]:
get_ipython().system('ls')

 phobert_fold1.pth   phobert_fold4.pth			 UIT-VSMEC
 phobert_fold2.pth   phobert_fold5.pth
 phobert_fold3.pth  'Screenshot 2023-06-01 000433.png'


Lệnh **`get_ipython().system('ls')`** được sử dụng trong môi trường Jupyter Notebook hoặc IPython để thực thi lệnh hệ thống **`ls`** (liệt kê các tệp tin và thư mục) và hiển thị kết quả trong ô output của Notebook.

## Download packages

In [ ]:
!pip install openpyxl


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


**`openpyxl`** là một thư viện Python được sử dụng để làm việc với các tệp Excel (**`.xlsx`**). Nó cung cấp các công cụ cho việc đọc, ghi và chỉnh sửa dữ liệu trong các tệp Excel, cho phép thao tác với các bảng tính, các ô và các công thức trong tệp Excel.

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Gói **`transformers`** là một thư viện Python phổ biến được phát triển bởi Hugging Face. Nó cung cấp các công cụ và mô hình để làm việc với xử lý ngôn ngữ tự nhiên (**`NLP`**) và học sâu (**`deep learning`**). Thư viện này cung cấp các mô hình nổi tiếng như **`BERT, GPT, RoBERTa`** và nhiều mô hình khác, và cung cấp các chức năng để tải, huấn luyện và sử dụng những mô hình này trong các tác vụ NLP.

## Import libraries

In [ ]:
import torch
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from gensim.utils import simple_preprocess
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix

import torch.nn as nn
from torch.optim import AdamW
from torch.utils.data import Dataset, DataLoader

from transformers import get_linear_schedule_with_warmup, AutoTokenizer, AutoModel, logging

import warnings
warnings.filterwarnings("ignore")

logging.set_verbosity_error()

- **`torch`**: Đây là thư viện PyTorch, một thư viện mã nguồn mở cho việc tính toán số học trên GPU và xây dựng các mô hình học sâu.
- **`numpy as np`**: Thư viện NumPy, được sử dụng để làm việc với mảng và ma trận số học.
- **`pandas as pd`**: Thư viện Pandas, được sử dụng để làm việc với dữ liệu dạng bảng và thực hiện các thao tác xử lý dữ liệu.
- **`seaborn as sns`**: Thư viện Seaborn, được sử dụng để tạo biểu đồ và trực quan hóa dữ liệu.
- **`matplotlib.pyplot as plt`**: Một phần của thư viện Matplotlib, được sử dụng để tạo biểu đồ và trực quan hóa dữ liệu.
- **`gensim.utils.simple_preprocess`**: Một phần của thư viện Gensim, được sử dụng để tiền xử lý văn bản và chuyển đổi văn bản thành các từ đơn giản.
- **`sklearn.model_selection.StratifiedKFold`**: Một phần của thư viện Scikit-learn, được sử dụng để thực hiện phân chia dữ liệu theo Stratified K-Fold cho việc cross-validation.
- **`sklearn.metrics.classification_report`**: Một phần của thư viện Scikit-learn, được sử dụng để tính toán báo cáo phân loại dựa trên các chỉ số như độ chính xác, độ phủ và f1-score.
- **`sklearn.metrics.confusion_matrix`**: Một phần của thư viện Scikit-learn, được sử dụng để tính toán ma trận nhầm lẫn (confusion matrix) cho các bài toán phân loại.
- **`torch.nn`**: Mô-đun trong thư viện PyTorch cho các lớp mô hình và các hàm mất mát (loss functions).
- **`torch.optim.AdamW`**: Một phần của thư viện PyTorch, cung cấp trình tối ưu hóa AdamW.
- **`torch.utils.data.Dataset, torch.utils.data.DataLoader`**: Các phần của thư viện PyTorch dùng để xây dựng và tải dữ liệu vào mô hình.
- **`transformers`**: Thư viện Hugging Face Transformers, được sử dụng để làm việc với các mô hình NLP đã được huấn luyện trước (pretrained models) như BERT, GPT, RoBERTa, và các hàm tiện ích liên quan.
- **`warnings`**: Một mô-đun trong Python để quản lý cảnh báo và thông báo trong quá trình chạy mã.

## Function definition

In [ ]:
def seed_everything(seed_value):
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    
    if torch.cuda.is_available(): 
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = True

Đoạn mã trên định nghĩa một hàm **`seed_everything`** để thiết lập các giá trị hạt giống (seed) cho việc tái lặp lại (reproducibility) quá trình huấn luyện mô hình. Dưới đây là ý nghĩa của từng dòng mã trong hàm:

- **`np.random.seed(seed_value)`**: Thiết lập giá trị hạt giống cho thư viện NumPy để tái lặp lại các phép ngẫu nhiên.
- **`torch.manual_seed(seed_value)`**: Thiết lập giá trị hạt giống cho thư viện PyTorch để tái lặp lại các phép ngẫu nhiên.
- **`if torch.cuda.is_available()`**: Kiểm tra xem có sử dụng GPU (CUDA) hay không.
- **`torch.cuda.manual_seed(seed_value)`**: Thiết lập giá trị hạt giống cho CUDA để tái lặp lại các phép ngẫu nhiên trên GPU.
- **`torch.cuda.manual_seed_all(seed_value)`**: Thiết lập giá trị hạt giống cho tất cả các thiết bị CUDA (nếu có).
- **`torch.backends.cudnn.deterministic = True`**: Đặt chế độ xác định (deterministic) cho việc sử dụng thư viện cuDNN trên GPU. Điều này đảm bảo kết quả của các phép tính trên GPU là nhất quán và tái lặp lại được.
- **`torch.backends.cudnn.benchmark = True`**: Đặt chế độ benchmark cho việc sử dụng thư viện cuDNN trên GPU. Điều này cho phép PyTorch tự động tinh chỉnh cấu hình cuDNN để tối ưu hóa hiệu suất.

Bằng cách thiết lập các giá trị hạt giống như trên, ta có thể đảm bảo rằng quá trình huấn luyện mô hình sẽ cho ra kết quả nhất quán và có thể tái lặp lại được khi chạy nhiều lần.

In [ ]:
seed_everything(86)

In [ ]:
def get_data(path):
    df = pd.read_excel(path, sheet_name='Sheet1')
    df.columns = ['index', 'Emotion', 'Sentence']
    df.drop(columns=['index'], inplace=True)
    return df

Hàm **`get_data`** trong đoạn mã trên nhận đầu vào là đường dẫn tới tệp Excel và trả về một dataframe chứa dữ liệu.

Các bước trong hàm **`get_data`** là như sau:

1. **`df = pd.read_excel(path, sheet_name='Sheet1')`**: Đọc tệp Excel từ đường dẫn **`path`** và lưu dữ liệu vào dataframe **`df`**. Tham số **`sheet_name='Sheet1'`** chỉ định rằng dữ liệu được đọc từ sheet có tên là 'Sheet1'.
2. **`df.columns = ['index', 'Emotion', 'Sentence']`**: Đặt tên cột cho dataframe **`df`** thành ['index', 'Emotion', 'Sentence']. Các cột này tương ứng với các cột trong tệp Excel.
3. **`df.drop(columns=['index'], inplace=True)`**: Xóa cột 'index' khỏi dataframe **`df`** vì không cần thiết.
4. **`return df`**: Trả về dataframe **`df`** đã xử lý.

In [ ]:
def train(model, criterion, optimizer, train_loader):
    model.train()
    losses = []
    correct = 0

    for data in train_loader:
        input_ids = data['input_ids'].to(device)
        attention_mask = data['attention_masks'].to(device)
        targets = data['targets'].to(device)

        optimizer.zero_grad()
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask
        )

        loss = criterion(outputs, targets)
        _, pred = torch.max(outputs, dim=1)

        correct += torch.sum(pred == targets)
        losses.append(loss.item())
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        lr_scheduler.step()

    print(f'Train Accuracy: {correct.double()/len(train_loader.dataset)} Loss: {np.mean(losses)}')

Hàm **`train`** được sử dụng để huấn luyện mô hình. Các tham số đầu vào bao gồm:

- **`model`**: Mô hình **`SentimentClassifier`** đã được khởi tạo.
- **`criterion`**: Hàm mất mát để tính toán độ lỗi (ví dụ: **`nn.CrossEntropyLoss`**).
- **`optimizer`**: Bộ tối ưu hóa được sử dụng để cập nhật trọng số của mô hình (ví dụ: **`AdamW`**).
- **`train_loader`**: **`DataLoader`** chứa dữ liệu huấn luyện.

Trong quá trình huấn luyện, các bước sau được thực hiện:

- Đặt mô hình vào chế độ huấn luyện bằng cách gọi **`model.train()`**.
- Khởi tạo biến **`losses`** để lưu trữ giá trị mất mát từ các batch.
- Khởi tạo biến **`correct`** để lưu trữ số lượng dự đoán chính xác.
- Với mỗi batch trong **`train_loader`**, thực hiện các bước sau:
  - Di chuyển dữ liệu vào GPU (nếu có) bằng cách gọi **`to(device)`**.
  - Xóa các gradient trước khi tính toán backward bằng **`optimizer.zero_grad()`**.
  - Đưa dữ liệu qua mô hình để có đầu ra (**`outputs`**).
  - Tính toán giá trị mất mát bằng cách so sánh đầu ra và nhãn thật (**`targets`**) thông qua **`criterion`**.
  - Dùng **`torch.max`** để lấy ra nhãn dự đoán có giá trị lớn nhất.
  - Cập nhật biến **`correct`** bằng cách tính toán số lượng dự đoán chính xác.
  - Lưu trữ giá trị mất mát vào biến **`losses`**.
  - Tính gradient và cập nhật trọng số mô hình thông qua **`loss.backward()`** và **`optimizer.step()`**.
  - Áp dụng quá trình điều chỉnh learning rate thông qua **`lr_scheduler.step()`** (nếu có).
  
Cuối cùng, hàm in ra thông tin về độ chính xác trên tập huấn luyện và giá trị trung bình của mất mát (**`np.mean(losses)`**).

In [ ]:
def eval(test_data = False):
    model.eval()
    losses = []
    correct = 0

    with torch.no_grad():
        data_loader = test_loader if test_data else valid_loader
        for data in data_loader:
            input_ids = data['input_ids'].to(device)
            attention_mask = data['attention_masks'].to(device)
            targets = data['targets'].to(device)

            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask
            )

            _, pred = torch.max(outputs, dim=1)

            loss = criterion(outputs, targets)
            correct += torch.sum(pred == targets)
            losses.append(loss.item())
    
    if test_data:
        print(f'Test Accuracy: {correct.double()/len(test_loader.dataset)} Loss: {np.mean(losses)}')
        return correct.double()/len(test_loader.dataset)
    else:
        print(f'Valid Accuracy: {correct.double()/len(valid_loader.dataset)} Loss: {np.mean(losses)}')
        return correct.double()/len(valid_loader.dataset)

Hàm **`eval`** được sử dụng để đánh giá hiệu suất của mô hình trên tập kiểm tra hoặc tập xác thực. Các tham số đầu vào bao gồm:

- **`test_data`**: Một cờ boolean xác định liệu ta đang đánh giá trên tập kiểm tra (**`True`**) hay tập xác thực (**`False`**).

Trong quá trình đánh giá, các bước sau được thực hiện:

- Đặt mô hình vào chế độ đánh giá bằng cách gọi **`model.eval()`**.
- Khởi tạo biến **`losses`** để lưu trữ giá trị mất mát từ các batch.
- Khởi tạo biến **`correct`** để lưu trữ số lượng dự đoán chính xác.
- Sử dụng **`torch.no_grad()`** để tắt tính toán gradient.
- Chọn **`data_loader`** tương ứng (từ **`test_loader`** hoặc **`valid_loader`**) dựa trên giá trị của **`test_data`**.
- Với mỗi batch trong **`data_loader`**, thực hiện các bước sau:
  - Di chuyển dữ liệu vào GPU (nếu có) bằng cách gọi **`to(device)`**.
  - Đưa dữ liệu qua mô hình để có đầu ra (**`outputs`**).
  - Dùng **`torch.max`** để lấy ra nhãn dự đoán có giá trị lớn nhất.
  - Tính toán giá trị mất mát bằng cách so sánh đầu ra và nhãn thật thông qua **`criterion`**.
  - Cập nhật biến **`correct`** bằng cách tính toán số lượng dự đoán chính xác.
  - Lưu trữ giá trị mất mát vào biến **`losses`**.

Cuối cùng, hàm in ra thông tin về độ chính xác trên tập kiểm tra hoặc tập xác thực và giá trị trung bình của mất mát (**`np.mean(losses)`**). Hàm trả về độ chính xác tính được trên tập kiểm tra hoặc tập xác thực.

In [ ]:
def prepare_loaders(df, fold):
    df_train = df[df.kfold != fold].reset_index(drop=True)
    df_valid = df[df.kfold == fold].reset_index(drop=True)
    
    train_dataset = SentimentDataset(df_train, tokenizer, max_len=120)
    valid_dataset = SentimentDataset(df_valid, tokenizer, max_len=120)
    
    train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=2)
    valid_loader = DataLoader(valid_dataset, batch_size=16, shuffle=True, num_workers=2)
    
    return train_loader, valid_loader

Hàm **`prepare_loaders`** được sử dụng để chuẩn bị các DataLoader cho việc huấn luyện và đánh giá trên mỗi fold. Các tham số đầu vào bao gồm:

- **`df`**: DataFrame chứa dữ liệu huấn luyện đã được gán fold.
- **`fold`**: Fold hiện tại đang được xử lý.

Các bước thực hiện trong hàm là:

- Tách DataFrame **`df`** thành **`df_train`** và **`df_valid`** dựa trên fold hiện tại.
- Tạo đối tượng **`train_dataset`** và **`valid_dataset`** từ **`df_train`** và **`df_valid`** sử dụng lớp **`SentimentDataset`**.
- Tạo DataLoader **`train_loader`** và **`valid_loader`** từ **`train_dataset`** và **`valid_dataset`** với các thông số:
  - **`batch_size=16`**: Số lượng mẫu trong mỗi batch.
  - **`shuffle=True`**: Xáo trộn dữ liệu trước khi tạo batch.
  - **`num_workers=2`**: Số luồng xử lý dữ liệu song song.

Cuối cùng, hàm trả về **`train_loader`** và **`valid_loader`** để sử dụng trong quá trình huấn luyện và đánh giá.

In [ ]:
def test(data_loader):
    models = []
    for fold in range(skf.n_splits):
        model = SentimentClassifier(n_classes=7)
        model.to(device)
        model.load_state_dict(torch.load(f'phobert_fold{fold+1}.pth', map_location=torch.device('cpu')))
        model.eval()
        models.append(model)

    texts = []
    predicts = []
    predict_probs = []
    real_values = []

    for data in data_loader:
        text = data['text']
        input_ids = data['input_ids'].to(device)
        attention_mask = data['attention_masks'].to(device)
        targets = data['targets'].to(device)

        total_outs = []
        for model in models:
            with torch.no_grad():
                outputs = model(
                    input_ids=input_ids,
                    attention_mask=attention_mask
                )
                total_outs.append(outputs)
        
        total_outs = torch.stack(total_outs)
        _, pred = torch.max(total_outs.mean(0), dim=1)
        texts.extend(text)
        predicts.extend(pred)
        predict_probs.extend(total_outs.mean(0))
        real_values.extend(targets)
    
    predicts = torch.stack(predicts).cpu()
    predict_probs = torch.stack(predict_probs).cpu()
    real_values = torch.stack(real_values).cpu()
    print(classification_report(real_values, predicts))
    return real_values, predicts

Hàm **`test`** được sử dụng để đánh giá mô hình trên dữ liệu kiểm tra hoặc dữ liệu mới. Các bước thực hiện trong hàm **`test`** là:

- Khởi tạo một danh sách **`models`** để lưu trữ các mô hình đã được huấn luyện trên các fold trước đó.
- Vòng lặp qua các fold để tải và tạo mô hình từ trạng thái đã lưu.
- Thiết lập mô hình ở chế độ đánh giá bằng cách gọi **`model.eval()`**.
- Tạo các danh sách để lưu trữ dữ liệu về văn bản, dự đoán, xác suất dự đoán và giá trị thực của các mẫu.
- Vòng lặp qua từng mẫu trong **`data_loader`**.
  - Lấy thông tin văn bản, đầu vào **`input_ids`**, **`attention_mask`** và giá trị thực **`targets`**.
  - Tạo danh sách **`total_outs`** để lưu trữ đầu ra của các mô hình trên từng fold.
  - Vòng lặp qua các mô hình trong **`models`**.
    - Sử dụng mô hình để tính toán đầu ra cho mẫu đầu vào.
    - Lưu trữ đầu ra vào danh sách **`total_outs`**.
  - Chuyển đổi danh sách **`total_outs`** thành tensor và tính toán dự đoán cuối cùng bằng cách lấy giá trị lớn nhất trung bình.
  - Gắn kết thông tin văn bản, dự đoán, xác suất dự đoán và giá trị thực vào các danh sách tương ứng.
- Chuyển đổi dự đoán, xác suất dự đoán và giá trị thực thành tensor trên CPU.
- In báo cáo phân loại bằng cách gọi **`classification_report`** từ **`sklearn.metrics`**.
- Trả về các tensor **`real_values`** (giá trị thực) và **`predicts`** (dự đoán).

In [ ]:
def check_wrong(real_values, predicts):
    wrong_arr = []
    wrong_label = []
    for i in range(len(predicts)):
        if predicts[i] != real_values[i]:
            wrong_arr.append(i)
            wrong_label.append(predicts[i])
    return wrong_arr, wrong_label

Hàm **`check_wrong`** được sử dụng để kiểm tra các trường hợp dự đoán sai của mô hình.

- Đầu vào của hàm là **`real_values`** (giá trị thực) và **`predicts`** (dự đoán).
- Hàm sẽ duyệt qua từng phần tử trong **`predicts`** và so sánh với giá trị tương ứng trong **`real_values`**.
- Nếu dự đoán không khớp với giá trị thực, vị trí của phần tử đó sẽ được thêm vào danh sách **`wrong_arr`**, và dự đoán sai sẽ được thêm vào danh sách **`wrong_label`**.
- Cuối cùng, hàm trả về **`wrong_arr`** (danh sách các vị trí dự đoán sai) và **`wrong_label`** (danh sách các dự đoán sai).

In [ ]:
def infer(text, tokenizer, max_len=120):
    encoded_review = tokenizer.encode_plus(
        text,
        max_length=max_len,
        truncation=True,
        add_special_tokens=True,
        padding='max_length',
        return_attention_mask=True,
        return_token_type_ids=False,
        return_tensors='pt',
    )

    input_ids = encoded_review['input_ids'].to(device)
    attention_mask = encoded_review['attention_mask'].to(device)

    output = model(input_ids, attention_mask)
    _, y_pred = torch.max(output, dim=1)

    print(f'Text: {text}')
    print(f'Sentiment: {class_names[y_pred]}')

Đoạn code trên định nghĩa một hàm **`infer`** để dự đoán cảm xúc của một câu văn mới bằng cách sử dụng mô hình đã được huấn luyện. 

Hàm **`infer`** nhận vào các đối số sau:
- **`text`**: Đây là câu văn cần dự đoán cảm xúc.
- **`tokenizer`**: Đối tượng tokenizer được sử dụng để mã hóa câu văn thành dạng phù hợp cho mô hình.
- **`max_len`**: Độ dài tối đa của câu văn sau khi mã hóa.

Trước tiên, câu văn **`text`** được mã hóa bằng cách sử dụng tokenizer. Quá trình mã hóa này bao gồm thêm các token đặc biệt vào câu văn, cắt ngắn hoặc bổ sung các token để đảm bảo độ dài của câu văn không vượt quá **`max_len`**, và tạo các tensor đại diện cho câu văn (bao gồm tensor **`input_ids`** và **`attention_mask`**).

Tiếp theo, câu văn được đưa vào mô hình để dự đoán cảm xúc. Mô hình trả về một tensor chứa xác suất dự đoán cho từng lớp cảm xúc. Bằng cách sử dụng hàm **`torch.max`**, ta lấy chỉ số của lớp có xác suất dự đoán cao nhất (**`y_pred`**).

Cuối cùng, hàm **`infer`** in ra câu văn ban đầu (**`text`**) và cảm xúc dự đoán tương ứng (**`class_names[y_pred]`**). **`class_names`** là danh sách các nhãn cảm xúc tương ứng với các lớp.

Bằng cách sử dụng hàm **`infer`**, ta có thể dễ dàng dự đoán cảm xúc của các câu văn mới bằng cách gọi hàm và truyền vào câu văn cần dự đoán.

## Class definition

In [ ]:
class SentimentDataset(Dataset):
    def __init__(self, df, tokenizer, max_len=120):
        self.df = df
        self.max_len = max_len
        self.tokenizer = tokenizer
    
    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        """
        To customize dataset, inherit from Dataset class and implement
        __len__ & __getitem__
        __getitem__ should return 
            data:
                input_ids
                attention_masks
                text
                targets
        """
        row = self.df.iloc[index]
        text, label = self.get_input_data(row)

        # Encode_plus will:
        # (1) split text into token
        # (2) Add the '[CLS]' and '[SEP]' token to the start and end
        # (3) Truncate/Pad sentence to max length
        # (4) Map token to their IDS
        # (5) Create attention mask
        # (6) Return a dictionary of outputs
        encoding = self.tokenizer.encode_plus(
            text,
            truncation=True,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_attention_mask=True,
            return_token_type_ids=False,
            return_tensors='pt',
        )
        
        return {
            'text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_masks': encoding['attention_mask'].flatten(),
            'targets': torch.tensor(label, dtype=torch.long),
        }


    def labelencoder(self,text):
        if text=='Enjoyment':
            return 0
        elif text=='Disgust':
            return 1
        elif text=='Sadness':
            return 2
        elif text=='Anger':
            return 3
        elif text=='Surprise':
            return 4
        elif text=='Fear':
            return 5
        else:
            return 6

    def get_input_data(self, row):
        # Preprocessing: {remove icon, special character, lower}
        text = row['Sentence']
        text = ' '.join(simple_preprocess(text))
        label = self.labelencoder(row['Emotion'])

        return text, label

Lớp **`SentimentDataset`** là một lớp con của **`torch.utils.data.Dataset`**, được sử dụng để tạo ra một tập dữ liệu tùy chỉnh cho nhiệm vụ phân loại cảm xúc (sentiment classification).

Các phương thức chính trong lớp **`SentimentDataset`** bao gồm:

- **`__init__(self, df, tokenizer, max_len=120)`**: Phương thức khởi tạo của lớp, nhận vào dataframe **`df`** chứa dữ liệu huấn luyện, tokenizer **`tokenizer`** và độ dài tối đa **`max_len`** của câu.
- **`__len__(self)`**: Phương thức trả về số lượng mẫu trong tập dữ liệu.
- **`__getitem__(self, index)`**: Phương thức trả về một mẫu dữ liệu tại vị trí **`index`**. Phương thức này thực hiện tiền xử lý văn bản, mã hóa văn bản thành các mã token và trả về một từ điển chứa thông tin của mẫu dữ liệu, bao gồm **`input_ids`** (mã token), **`attention_masks`** (mặt nạ chú ý), **`text`** (văn bản gốc) và **`targets`** (nhãn).
- **`labelencoder(self, text)`**: Phương thức chuyển đổi nhãn từ dạng văn bản sang số nguyên. Các nhãn được mã hóa như sau: **`Enjoyment`**: 0, **`Disgust`**: 1, **`Sadness`**: 2, **`Anger`**: 3, **`Surprise`**: 4, **`Fear`**: 5, **`Khác`**: 6.
- **`get_input_data(self, row)`**: Phương thức thực hiện xử lý dữ liệu đầu vào từ một hàng dữ liệu trong dataframe **`df`**. Nó loại bỏ các ký tự đặc biệt, chuyển đổi văn bản thành dạng lowercase và trả về văn bản và nhãn tương ứng.

Lớp **`SentimentDataset`** cho phép tạo ra các đối tượng tập dữ liệu tuỳ chỉnh để huấn luyện và đánh giá mô hình phân loại cảm xúc.

In [ ]:
class SentimentClassifier(nn.Module):
    def __init__(self, n_classes):
        super(SentimentClassifier, self).__init__()
        self.bert = AutoModel.from_pretrained("vinai/phobert-base")
        self.drop = nn.Dropout(p=0.3)
        self.fc = nn.Linear(self.bert.config.hidden_size, n_classes)
        nn.init.normal_(self.fc.weight, std=0.02)
        nn.init.normal_(self.fc.bias, 0)

    def forward(self, input_ids, attention_mask):
        last_hidden_state, output = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            return_dict=False # Dropout will errors if without this
        )

        x = self.drop(output)
        x = self.fc(x)
        return x

Lớp **`SentimentClassifier`** là một mô hình phân loại cảm xúc dựa trên kiến trúc của RoBERTa. Cụ thể:

- **`n_classes`**: Số lượng lớp đầu ra (số lượng cảm xúc khác nhau).

Phương thức **`__init__`**:
- Khởi tạo mô hình RoBERTa (**`self.bert`**) từ pretrained weights của bộ mã hóa PhoBERT.
- Áp dụng Dropout với xác suất 0.3 (**`self.drop`**) để tránh overfitting.
- Tạo một lớp tuyến tính (**`self.fc`**) với kích thước đầu ra bằng **`n_classes`** và khởi tạo các trọng số ngẫu nhiên theo phân phối chuẩn.

Phương thức **`forward`**:
- Đầu vào của mô hình là **`input_ids`** (chuỗi các mã token) và **`attention_mask`** (mảng chỉ định phần tử thực tế và padding trong câu).
- Mô hình RoBERTa được áp dụng cho đầu vào và trả về kết quả cuối cùng và hidden state (**`last_hidden_state`**, **`output`**).
- Đầu ra từ RoBERTa được truyền qua Dropout layer và sau đó thông qua một lớp tuyến tính để đưa ra dự đoán cho từng lớp.

Lớp **`SentimentClassifier`** là một lớp con của lớp **`nn.Module`** trong PyTorch và triển khai phương thức **`forward`** để thực hiện feedforward qua mô hình.

## Parameter definition

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
EPOCHS = 6
N_SPLITS = 5

- **`device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')`**: Dòng này xác định thiết bị tính toán mà PyTorch sẽ sử dụng. Nếu CUDA (GPU) khả dụng, thiết bị sẽ được đặt là 'cuda:0' (thiết bị GPU đầu tiên), ngược lại nó sẽ được đặt là 'cpu' (thiết bị CPU).
- **`EPOCHS = 6`**: Biến EPOCHS định nghĩa số lượng epochs (vòng lặp huấn luyện) mà mô hình sẽ được huấn luyện qua.
- **`N_SPLITS = 5`**: Biến N_SPLITS định nghĩa số lượng phân chia (splits) trong quá trình cross-validation (kiểm định chéo). Đây là số lượng phần chia dữ liệu huấn luyện và kiểm tra mà mô hình sẽ được huấn luyện và đánh giá trên. Trong trường hợp này, có tổng cộng 5 phân chia.

# 2. Load data

## Dataset Introduction

In [ ]:
train_df = get_data('/content/drive/MyDrive/Colab Notebooks/PhoBERT_Emotion/UIT-VSMEC/train_nor_811.xlsx')
valid_df = get_data('/content/drive/MyDrive/Colab Notebooks/PhoBERT_Emotion/UIT-VSMEC/valid_nor_811.xlsx')
test_df = get_data('/content/drive/MyDrive/Colab Notebooks/PhoBERT_Emotion/UIT-VSMEC/test_nor_811.xlsx')

In [ ]:
train_df = pd.concat([train_df, valid_df], ignore_index=True)
skf = StratifiedKFold(n_splits=N_SPLITS)

for fold, (_, val_) in enumerate(skf.split(X=train_df, y=train_df.Emotion)):
    train_df.loc[val_, "kfold"] = fold

Đoạn mã trên thực hiện quá trình chia dữ liệu thành các phần chia kiểm định sử dụng StratifiedKFold.

Các bước trong đoạn mã là như sau:

1. **`train_df = pd.concat([train_df, valid_df], ignore_index=True)`**: Kết hợp dataframe train_df và valid_df lại thành một dataframe duy nhất. Tham số **`ignore_index=True`** được sử dụng để đặt lại chỉ mục của các dòng trong dataframe kết hợp.
2. **`skf = StratifiedKFold(n_splits=N_SPLITS)`**: Khởi tạo một đối tượng StratifiedKFold với số lượng phần chia kiểm định là N_SPLITS.
3. **`for fold, (_, val_) in enumerate(skf.split(X=train_df, y=train_df.Emotion))`**: Sử dụng vòng lặp for để duyệt qua các fold được tạo bởi StratifiedKFold. Biến fold đại diện cho số thứ tự của fold, còn (_, val_) đại diện cho chỉ mục của các dữ liệu huấn luyện và dữ liệu kiểm định trong fold hiện tại.
4. **`train_df.loc[val_, "kfold"] = fold`**: Đánh dấu các dữ liệu có chỉ mục là val_ trong cột "kfold" của dataframe train_df với giá trị của fold hiện tại.

# 4. Tokenization using PhoBERT

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base", use_fast=False)

Đoạn mã trên sử dụng thư viện **`transformers`** để tải và khởi tạo một tokenizer từ mô hình **`vinai/phobert-base`**.

Trong đó:
- **`"vinai/phobert-base"`** là tên của mô hình PhoBERT được cung cấp bởi VinAI Research.
- **`use_fast=False`** được sử dụng để tải tokenizer theo chế độ không sử dụng các cải tiến nhanh của Hugging Face. Mặc định là **`True`**, nhưng khi chọn **`False`**, việc tải tokenizer có thể mất nhiều thời gian hơn nhưng sẽ tiết kiệm bộ nhớ.

BERT works with fixed-length sequences. We’ll use a simple strategy to choose the max length.

# 5. Inference

In [ ]:
model.load_state_dict(torch.load(f'phobert_fold5.pth', map_location=torch.device('cpu')))

In [ ]:
infer('Thật là vui vì cuối cùng đã hoàn thành công việc', tokenizer)

Text: Thật là vui vì cuối cùng đã hoàn thành công việc
Sentiment: Enjoyment


In [ ]:
infer('Rớt môn rồi, tôi buồn quá', tokenizer)

Text: Rớt môn rồi, tôi buồn quá
Sentiment: Sadness


In [ ]:
infer('Tại sao lại đến trễ, thật bực mình', tokenizer)

Text: Tại sao lại đến trễ, thật bực mình
Sentiment: Disgust


In [ ]:
infer('Ồ! ở đó có nhiều vàng vậy hả?', tokenizer)

Text: Ồ! ở đó có nhiều vàng vậy hả?
Sentiment: Surprise


In [ ]:
infer('Cái đó nhìn ghê quá!', tokenizer)

In [ ]:
infer('Cái đó nhìn ghê quá!', tokenizer)

Text: Cái đó nhìn ghê quá!
Sentiment: Disgust


In [ ]:
infer('Bộ phim này chán quá', tokenizer)

Text: Bộ phim này chán quá
Sentiment: Disgust


In [ ]:
infer('Em yêu Hệ thống', tokenizer)

Text: Em yêu Hệ thống
Sentiment: Enjoyment


In [ ]:
infer('Tôi không hiểu bộ phim đang nói về vấn đề gì', tokenizer)

Text: Tôi không hiểu bộ phim đang nói về vấn đề gì
Sentiment: Other
